[//]: # ( Botanical Sciences Notebook Template )

[//]: # ( This template provides a standardized structure for botanical research notebooks )

[//]: # ( License: MIT License )

[//]: # ( Repository: https://github.com/outobecca/botanical-colabs )

[//]: # ( Date: 2025-11-04 )

[//]: # ( Creator: Your Name )

# 🌿 [Your Notebook Title]
**Version 1.0** | Created: YYYY-MM-DD | Author: [Your Name]

## 📋 Overview

**Purpose:** [Brief description of what this notebook does]

**Research Question/Goal:** [What question does this notebook answer or what problem does it solve?]

### 🎯 Use Cases
- [Use case 1]
- [Use case 2]
- [Use case 3]

### 📊 Data Sources

This notebook utilizes the following data sources:

| Data Source | Type | API Key | Information | License |
|------------|------|---------|-------------|----------|
| **[Source 1]** | Open/Paid | Required/Not required | [What data] | [License] |
| **[Source 2]** | Open/Paid | Required/Not required | [What data] | [License] |

### 🔑 Required API Keys

Links to obtain API keys (if needed):

- [API Name](https://link-to-api.com) — [Purpose]

### ⚠️ Important Notes
- [Any important warnings or limitations]
- Data is fetched in real-time, so availability may vary
- Always verify information from primary sources for scientific use
- Respect API rate limits and terms of service

## 📚 Background & Methodology

### Scientific Context
[Provide scientific background for your analysis]

### Methodology
[Explain your approach and methods]

### Expected Outputs
- [Output 1]
- [Output 2]
- [Output 3]

## ⚙️ Step 1: Installation and Configuration

### Instructions

1. **Install libraries** — Run the code below to install required Python libraries
2. **Add API keys** — Use Colab's **Secrets** feature (key icon on the left)
   - `API_KEY_NAME` — [Purpose]
3. **Configure parameters** — Set your input parameters below
4. **Save settings** — Press the save button

### 💡 Tips
- [Helpful tip 1]
- [Helpful tip 2]
- API keys are stored securely in Colab Secrets

In [ ]:
# ============================================================================
# STEP 1.1: Library Installation and Import
# ============================================================================
"""
Installs required Python libraries and imports them.
This cell should be run first.
"""

# Installation (run once)
# Add your required packages here
!pip install -q requests pandas numpy matplotlib seaborn ipywidgets

# Library imports
from typing import Dict, Optional, List, Any, Tuple
from IPython.display import display, Markdown, Image, HTML
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
import warnings
from datetime import datetime
import json

# Colab-specific import
try:
    from google.colab import userdata
    COLAB_ENV = True
    print("✅ Google Colab environment detected")
except ImportError:
    COLAB_ENV = False
    print("⚠️ Not in Colab environment. API keys must be set manually.")
    # Create mock userdata class for local use
    class MockUserData:
        def get(self, key: str) -> str:
            import os
            return os.environ.get(key, '')
    userdata = MockUserData()

# Configure plotting style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

# Reduce warnings
warnings.filterwarnings('ignore')

print(f"📅 Run at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("✅ Libraries imported successfully\n")

In [ ]:
# ============================================================================
# STEP 1.2: User Interface and Settings
# ============================================================================
"""
Creates an interactive user interface for parameter configuration.
"""

# Global variables
input_parameter: str = ""
API_KEY: str = ""
SETTINGS: Dict[str, bool] = {}

# UI components
input_widget = widgets.Text(
    value=globals().get("input_parameter", ""),
    placeholder="Enter your input here",
    description="📝 Input:",
    disabled=False,
    layout=widgets.Layout(width='500px'),
    style={'description_width': '100px'}
)

# Checkbox for options
option1_widget = widgets.Checkbox(
    value=True,
    description="Option 1",
    style={'description_width': '200px'}
)

option2_widget = widgets.Checkbox(
    value=True,
    description="Option 2",
    style={'description_width': '200px'}
)

save_button = widgets.Button(
    description="💾 Save Settings",
    button_style="success",
    icon="check",
    layout=widgets.Layout(width='200px')
)

status_output = widgets.Output()

def update_settings(_=None) -> None:
    """
    Updates global variables from UI values and checks API keys.
    
    Reads API keys securely from Colab Secrets.
    """
    global input_parameter, API_KEY, SETTINGS
    
    input_parameter = input_widget.value.strip()
    
    # Fetch API keys securely
    def get_api_key(key_name: str) -> str:
        """Fetches API key securely."""
        try:
            if COLAB_ENV:
                return userdata.get(key_name)
            else:
                return ""
        except Exception:
            return ""
    
    API_KEY = get_api_key('YOUR_API_KEY_NAME')
    
    # Update settings
    SETTINGS = {
        "option1": option1_widget.value,
        "option2": option2_widget.value,
    }
    
    # Show status
    with status_output:
        status_output.clear_output()
        if not input_parameter:
            print("⚠️ Enter your input parameter")
        else:
            print(f"✅ Settings updated: {input_parameter}")
            print(f"\n📋 API Key Status:")
            print(f"  API Key: {'✓ Set' if API_KEY else '✗ Missing'}")
            
            if not API_KEY:
                print(f"\n⚠️ Add API key to Colab Secrets for full functionality")

# Connect button to function
save_button.on_click(update_settings)

# Initialize settings
update_settings()

# Create UI
panel = widgets.VBox([
    widgets.HTML("<h3>⚙️ Configuration</h3>"),
    input_widget,
    widgets.HTML("<h3>🔧 Options</h3>"),
    widgets.HTML("<p><i>Select your analysis options</i></p>"),
    widgets.VBox([
        option1_widget,
        option2_widget
    ]),
    widgets.HTML("<br>"),
    widgets.HBox([save_button]),
    status_output
], layout=widgets.Layout(padding='20px', border='2px solid #ddd', border_radius='10px'))

display(panel)

## 🔧 Step 2: Helper Functions

These functions include error handling and data validation.

In [ ]:
# ============================================================================
# Helper Functions for Data Retrieval and Validation
# ============================================================================
"""
Contains general-purpose helper functions for API calls, error handling,
and data validation.
"""

def safe_api_call(url: str, params: Optional[Dict[str, Any]] = None, 
                  timeout: int = 15, method: str = 'GET') -> Optional[Dict[str, Any]]:
    """
    Safe API call with error handling.
    
    Args:
        url: API endpoint
        params: Query parameters
        timeout: Timeout in seconds
        method: HTTP method (GET or POST)
    
    Returns:
        JSON response as dictionary or None on error
    """
    try:
        if method.upper() == 'GET':
            response = requests.get(url, params=params, timeout=timeout)
        else:
            response = requests.post(url, json=params, timeout=timeout)
        
        response.raise_for_status()
        return response.json()
    except requests.exceptions.Timeout:
        print(f"⏱️ Timeout: {url}")
        return None
    except requests.exceptions.RequestException as e:
        print(f"❌ Network error: {str(e)[:100]}")
        return None
    except ValueError:
        print(f"❌ Invalid JSON response")
        return None

def validate_input(input_value: str) -> bool:
    """
    Validates input parameter.
    
    Args:
        input_value: Value to validate
    
    Returns:
        True if valid, False otherwise
    """
    if not input_value or len(input_value) < 2:
        print("⚠️ Input value is too short or empty")
        return False
    return True

def format_data_for_display(data: Optional[Dict[str, Any]], 
                            source_name: str) -> pd.DataFrame:
    """
    Formats data as Pandas DataFrame for display.
    
    Args:
        data: Data to display
        source_name: Data source name
    
    Returns:
        DataFrame for display
    """
    if not data:
        return pd.DataFrame({"Field": [f"{source_name} data"], "Value": ["Not available"]})
    
    rows = []
    for key, value in data.items():
        if value and value != "N/A":
            rows.append({"Field": key, "Value": str(value)})
    
    return pd.DataFrame(rows) if rows else pd.DataFrame({"Field": [f"{source_name} data"], "Value": ["Not available"]})

def create_citation(source: str, date: str = None) -> str:
    """
    Creates a citation for a data source.
    
    Args:
        source: Data source name
        date: Query date
    
    Returns:
        Formatted citation
    """
    if date is None:
        date = datetime.now().strftime('%Y-%m-%d')
    
    # Add your data source citations here
    citations = {
        "ExampleAPI": f"Example API ({date}). https://example.com",
    }
    
    return citations.get(source, f"{source} ({date})")

print("✅ Helper functions defined")

## 📡 Step 3: Data Fetching Functions

These functions retrieve data from various APIs or data sources.

In [ ]:
# ============================================================================
# Data Fetching Functions
# ============================================================================
"""
Functions for retrieving data from external sources.

Data Sources:
- [Source 1] - [URL]
- [Source 2] - [URL]
"""

def fetch_example_data(query: str, api_key: str = None) -> Optional[Dict[str, Any]]:
    """
    Fetches example data from an API.
    
    Args:
        query: Search query
        api_key: Optional API key
    
    Returns:
        Dictionary with data or None
        
    Source:
        Example API - https://example.com
        License: [License type]
    """
    # Example API endpoint
    url = "https://api.example.com/v1/search"
    params = {"q": query}
    
    if api_key:
        params["api_key"] = api_key
    
    data = safe_api_call(url, params)
    
    if not data:
        print(f"❌ No data found for: {query}")
        return None
    
    # Parse and format the response
    result = {
        "Field1": data.get("field1", "N/A"),
        "Field2": data.get("field2", "N/A"),
        "Field3": data.get("field3", "N/A"),
    }
    
    print(f"✅ Data retrieved successfully")
    return result

# Add more data fetching functions as needed

print("✅ Data fetching functions defined")

## 🚀 Step 4: Execute Data Collection

Run this cell to fetch data from all selected sources.

In [ ]:
# ============================================================================
# Main Execution: Collect Data from All Sources
# ============================================================================
"""
Executes data collection from all configured sources and compiles results.
"""

if not input_parameter:
    print("❌ ERROR: Enter input parameter in Step 1 and press 'Save Settings'")
else:
    # Validate input
    if not validate_input(input_parameter):
        print("⚠️ WARNING: Input may not be in correct format. Continuing anyway...")
    
    print(f"\n📊 Collecting data for: {input_parameter}")
    print("=" * 60)
    
    # Initialize result storage
    all_data = {}
    
    # 1. Fetch from Source 1
    if SETTINGS.get("option1", True):
        print("\n📍 Fetching from Source 1...")
        source1_data = fetch_example_data(input_parameter, API_KEY)
        if source1_data:
            all_data.update(source1_data)
    else:
        print("ℹ️ Source 1 skipped per settings")
        source1_data = None
    
    # 2. Fetch from Source 2
    if SETTINGS.get("option2", True):
        print("\n📍 Fetching from Source 2...")
        # Add your data fetching logic here
        source2_data = None
    else:
        print("ℹ️ Source 2 skipped per settings")
        source2_data = None
    
    print("\n" + "=" * 60)
    print("✅ Data collection complete!")
    print("=" * 60)
    
    # Create DataFrame for display
    results_df = pd.DataFrame([
        {"Field": key, "Value": value} 
        for key, value in all_data.items() 
        if value and value != "N/A"
    ])
    
    if results_df.empty:
        results_df = pd.DataFrame([{"Field": "Status", "Value": "No data available from selected sources"}])

## 📊 Step 5: Data Analysis & Visualization

Analyze and visualize the collected data.

In [ ]:
# ============================================================================
# Data Analysis and Visualization
# ============================================================================
"""
Analyzes collected data and creates visualizations.
"""

# Display header
display(Markdown(f"# 📊 Analysis Results: **{input_parameter}**"))
display(Markdown("***"))

# Display data table
if not results_df.empty:
    display(Markdown("## 📋 Collected Data"))
    display(results_df.style.hide(axis='index'))
else:
    display(Markdown("⚠️ No data could be retrieved."))

# Example visualization
display(Markdown("\n## 📈 Visualizations"))

# Create sample plot (replace with your actual visualization)
fig, ax = plt.subplots(figsize=(10, 6))

# Example bar plot
# Replace this with your actual data visualization
example_data = {'Category A': 10, 'Category B': 15, 'Category C': 7}
ax.bar(example_data.keys(), example_data.values(), color='forestgreen', alpha=0.7)
ax.set_xlabel('Categories', fontsize=12)
ax.set_ylabel('Values', fontsize=12)
ax.set_title('Example Visualization', fontsize=14, fontweight='bold')
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

# Statistical summary
display(Markdown("\n## 📐 Statistical Summary"))
display(Markdown("""
**Key Findings:**
- [Finding 1]
- [Finding 2]
- [Finding 3]
"""))

print("\n✅ Analysis complete!")

## 📚 Step 6: Data Sources & Citations

Document all data sources and create proper citations.

In [ ]:
# ============================================================================
# Citations and Data Provenance
# ============================================================================
"""
Creates citations for all data sources used in this analysis.
"""

display(Markdown(f"\n## 📚 Data Sources & Citations"))
display(Markdown(f"Data retrieved on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"))

citations = []

# Add citations for sources that were actually used
if source1_data:
    citations.append(create_citation("ExampleAPI"))

# Display citations
if citations:
    display(Markdown("\n**Please cite:**"))
    for citation in citations:
        display(Markdown(f"- {citation}"))
else:
    display(Markdown("No data sources were used."))

# Notebook citation
display(Markdown(f"""
\n**This notebook:**
```
[Your Name]. ({datetime.now().year}). [Notebook Title].
Botanical Sciences Colab Notebooks.
https://github.com/outobecca/botanical-colabs
```
"""))

---

## 📝 Notes and Best Practices

### ✅ Data Quality
- Always verify information from multiple sources
- Check data freshness - API data is dynamic
- Document any data transformations or cleaning steps

### 🔬 Scientific Use
- Cite all data sources in publications
- Note API version and query date
- Be aware of data limitations and biases
- Keep detailed methodology notes

### 🐛 Troubleshooting

**No data returned:**
- Check input parameter format
- Verify API keys in Colab Secrets
- Check API rate limits

**API errors:**
- Verify API key validity
- Check rate limits not exceeded
- Verify network connectivity

**Visualization issues:**
- Ensure data is in correct format
- Check for missing or null values
- Verify plot parameters

### 🔗 Useful Resources
- [Repository](https://github.com/outobecca/botanical-colabs)
- [API Setup Guide](../API_SETUP.md)
- [Contributing Guidelines](../CONTRIBUTING.md)

---

## 📄 License & Attribution

**Code License:** MIT License

**Data Sources:** Each data source has its own license. See citations above.

**Repository:** https://github.com/outobecca/botanical-colabs

---

**Version:** 1.0  
**Last Updated:** [Date]  
**Created by:** [Your Name]  

🌿 Happy researching! 🔬